<a href="https://colab.research.google.com/github/piltom/widget_sketch_recon/blob/main/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconocimiento de widgets a mano alzada para generar UIs a partir de un borrador
## Big Picture
Este trabajo es una parte de un proyecto más grande, en el cual se intenta generar una descripción de una interfaz de usuario a partir de una foto de un sketch en papel/pizarra. Con esta descripción se puede después generar código para algún lenguaje/framwork en particular (ReactJS, Qt, Gtk, etc.)
Un ejemplo de un par de sketch e interfaz es el siguiente:

![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/ideal_example/sketch1.jpg)
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/ideal_example/final_result1.jpg)


##Este trabajo
En este trabajo, se asume que la imagen original ya fue segmentada en elementos únicos, los cuales deben ser clasificados en categorías. Los elementos posibles por el momento son:

###Checkbox:
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/checkbox.png)
###Input
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/input.png)

###Pushbutton:
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/pushbutton.png)

###Radiobutton:
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/radio1.png)
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/radio2.png)

###Round Image:
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/roundimg.png)

###Square Image:
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/squareimg.png)

###Slider:
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/slider.png)

###Switch:
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/switch1.png)
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/switch2.png)
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/switch3.png)

###Text:
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/text1.png)
![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/sample_elements/text2.png)

#Generación del dataset
El dataset fue dibujado sobre hojas cuadriculadas con lapicera azul, en formato de "sprite sheet" (rectángulos de un mismo tamaño contiguos). El fondo cuadriculado seguramente sea el "peor caso" de ruido de fondo posible, ya que alternativamente podría ser una hoja rayada o una pizarra. Se dibujaron diversas variaciones de cada elemento, utilizando distintos trazos (continuos, lineas, figuras que no llegan a cerrar, partes pintadas, etc.). En [MEGA](https://mega.nz/file/IUhz3KoB#ZqoRcKabLWs6Y8_xd8Rqv8WJIc1NzcchaOsEAV8c6n0) se encuentran tanto los "sprite sheets" como cada "tile" separado.

#Procesamiento y extracción de features
##Preprocesamiento
Antes de poder extraer features de las imágenes, se debe ajustar el tamaño de las imágenes y filtrar el fondo cuadriculado. Para esto se utiliza el script [adecuate.py](https://github.com/piltom/widget_sketch_recon/blob/main/util_test_scripts/adecuate.py), que cambia el tamaño de las imágenes para que tengan 256 pixeles de ancho y aplica un [threshold Li](https://scikit-image.org/docs/dev/auto_examples/developers/plot_threshold_li.html) para filtrar el fondo.

Un ejemplo del resultado de esta adecuación es:

![](https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/img/pushbutton/tile008_p.png)

**Nota: Posiblemente se podría reducir el tamaño de las imágenes aún más, cortando las partes negras en los bordes.**

##Features
Para este trabajo se consideraron tres métodos de extracción de features diferentes, con los cuales se generaron diversos dataframes alterando parámetros y se analizó la efectividad de los mismos usando un Support Vector Classifier y un Random Forest Classifier. Para generar los dataframes se utilizaron los scripts [create_widget_dataset.py](https://github.com/piltom/widget_sketch_recon/blob/main/create_widget_dataset.py) (ORB), [create_widget_dataset_hough.py](https://github.com/piltom/widget_sketch_recon/blob/main/create_widget_dataset_hough.py) (Transformada de Hough), [create_widget_dataset_houghncross.py](https://github.com/piltom/widget_sketch_recon/blob/main/create_widget_dataset_houghncross.py) (Transformada de Hough y cruce de lineas en centroide)


In [4]:
!mkdir dataroot 
!wget -P dataroot https://github.com/piltom/widget_sketch_recon/raw/main/dataset_img_ORB_200.pkl
!wget -P dataroot https://github.com/piltom/widget_sketch_recon/raw/main/dataset_img_ORB50.pkl
!wget -P dataroot https://github.com/piltom/widget_sketch_recon/raw/main/dataset_img_hough5cross_fullsize.pkl
!wget -P dataroot https://github.com/piltom/widget_sketch_recon/raw/main/dataset_img_hough7_fullsize.pkl
!wget -P dataroot https://github.com/piltom/widget_sketch_recon/raw/main/dataset_img_hough7_8thsize.pkl
!wget -P dataroot https://github.com/piltom/widget_sketch_recon/raw/main/dataset_img_hough5_doublefullsize.pkl
!wget -P dataroot https://github.com/piltom/widget_sketch_recon/raw/main/dataset_img_hough5_fullsize.pkl
!pip3 install pickle5

mkdir: cannot create directory ‘dataroot’: File exists
--2021-01-01 15:49:52--  https://github.com/piltom/widget_sketch_recon/raw/main/dataset_img_ORB_200.pkl
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/dataset_img_ORB_200.pkl [following]
--2021-01-01 15:49:53--  https://raw.githubusercontent.com/piltom/widget_sketch_recon/main/dataset_img_ORB_200.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5449402 (5.2M) [application/octet-stream]
Saving to: ‘dataroot/dataset_img_ORB_200.pkl.1’

dataset_img_ORB_200 100%[===================>]   5.20M  21.0MB/s    in 0.2s   

###ORB
Oriented FAST and Rotated Brief es un detector de features locales robusto que se utiliza frecuentemente en tareas de reconocimiento de objetos y reconstrucción 3D. Es robusto frente a rotaciones y cambios de escala, por lo que es muy útil cuando se tratan objetos 3D tomados desde distintos ángulos.

Para este caso se crearon dos datasets, uno con 50 keypoints y otro con 200. El resultado obtenido se muestra a continuación, seguido de una explicación de su falla para este caso.

In [6]:
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle5 as pickle

classifier = svm.LinearSVC()
with open("dataroot/dataset_img_ORB50.pkl", "rb") as fh:
  dataset_df = pickle.load(fh)
#dataset_df=pd.read_pickle("dataroot/dataset_img_ORB50.pkl").dropna(axis=1)
print(dataset_df)
X=dataset_df.drop(['target', 'filename'], axis=1)
y=dataset_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=11, stratify=y)

# Aprenda a clasificar el conjunto de entrenamiento
classifier.fit(X_train, y_train)
y_predicted = classifier.predict(X_test)
conf_mat=metrics.confusion_matrix(y_test, y_predicted)
print(conf_mat)
print("Classification report for classifier {}\n{}\n".format(classifier, metrics.classification_report(y_test, y_predicted)))


          filename    target      2      3  ...  12798  12799  12800  12801
0    tile018_p.png  checkbox  False  False  ...  False  False  False  False
1    tile061_p.png  checkbox  False  False  ...  False  False  False  False
2    tile040_p.png  checkbox  False  False  ...  False  False  False  False
3    tile037_p.png  checkbox  False  False  ...   True   True  False  False
4    tile049_p.png  checkbox  False  False  ...  False  False   True  False
..             ...       ...    ...    ...  ...    ...    ...    ...    ...
414  tile012_p.png      text  False  False  ...  False   True  False  False
415  tile019_p.png      text  False  False  ...  False  False  False  False
416  tile045_p.png      text  False  False  ...  False  False  False  False
417  tile001_p.png      text  False  False  ...  False  False  False  False
418  tile010_p.png      text  False  False  ...  False  False  False  False

[419 rows x 12802 columns]
[[ 7  2  0  0  0  2  0  1  4]
 [ 0 13  1  1  0  0  0  1  0]
